<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/8_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

如果na值我也想排名呢? 就放最後排成最後的名次

In [ ]:
import pandas as pd
import numpy as np

# 示例数据
data = {
    'col1': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'D'],
    'col2': ['X', 'X', '', 'Y', 'Y', '', 'Z', 'Z', '', ''],
    'col3': [1, 2, np.nan, 1, 2, 3, 1, 2, np.nan, 4],
    'col4': [10, 12, 14, 10, 11, 13, 9, 10, 15, 16]
}
df = pd.DataFrame(data)

# 按 col3 值由大至小排序，将 NaN 值放在末尾
df_sorted = df.sort_values(by='col3', ascending=False, na_position='last')

# 计算 col3 的排名，NaN 值排名在最后
df_sorted['rank'] = df_sorted['col3'].rank(method='min', ascending=False, na_option='bottom')
# 对 NaN 值进行排名
nan_count = df_sorted['col3'].isna().sum()
df_sorted.loc[df_sorted['col3'].isna(), 'rank'] = range(len(df_sorted) - nan_count + 1, len(df_sorted) + 1)

# 打印结果
print(df_sorted)


In [ ]:
  col1 col2  col3  col4  rank
9    D        4.0    16   1.0
5    B        3.0    13   2.0
1    A    X   2.0    12   3.0
4    B    Y   2.0    11   3.0
7    C    Z   2.0    10   3.0
0    A    X   1.0    10   6.0
3    B    Y   1.0    10   6.0
6    C    Z   1.0     9   6.0
2    A        NaN    14   9.0
8    C        NaN    15  10.0


1. sort_values 中的 na_position='last' 将 NaN 值放在排序结果的末尾。
2. rank 方法中的 na_option='bottom' 表示 NaN 值在排名中不参与计算。
3. 对于 NaN 值，使用 range(non_nan_count + 1, len(df_sorted) + 1) 手动指定排名。